In [1]:
import fitz  # PyMuPDF
import os

pdf_folder = 'indirilenler'
pdf_texts = {}

for filename in os.listdir(pdf_folder):
    if filename.endswith('.pdf'):
        filepath = os.path.join(pdf_folder, filename)
        pdf_document = fitz.open(filepath)
        text = ""
        for page in pdf_document:
            text += page.get_text()
        pdf_texts[filename] = text

print(pdf_texts.keys())  # Hangi dosyaları okuduğumuzu görebilirsin


dict_keys(['Elektrik Piyasası Kanunu (6446).pdf', 'ELEKTRİK PİYASASI DENGELEME VE UZLAŞTIRMA YÖNETMELİĞİ.pdf', 'ELEKTRİK PİYASASI LİSANS YÖNETMELİĞİ.pdf', 'ELEKTRİK PİYASASI TARİFELER YÖNETMELİĞİ.pdf', 'ELEKTRİK PİYASASI TÜKETİCİ HİZMETLERİ YÖNETMELİĞİ.pdf', 'Teminat-Hesaplama-Yontemi-10.01.2025-1-1.pdf'])


In [2]:
# Her PDF dosyasının ilk 500 karakterini ekrana yazalım
for filename, text in pdf_texts.items():
    print(f"\n==== {filename} ====")
    print(text[:500])  # İlk 500 karakteri gösteriyoruz


==== Elektrik Piyasası Kanunu (6446).pdf ====
ELEKTRİK PİYASASI KANUNU 
  
Kanun Numarası                    : 6446 
Kabul Tarihi                           : 14/3/2013 
Yayımlandığı Resmî Gazete   : Tarih  : 30/3/2013        Sayı  : 28603 
Yayımlandığı Düstur              : Tertip : 5                      Cilt   : 53 
  
BİRİNCİ BÖLÜM 
Amaç, Kapsam ve Tanımlar 
Amaç 
MADDE 1 – (1) Bu Kanunun amacı; elektriğin yeterli, kaliteli, sürekli, düşük 
maliyetli ve çevreyle uyumlu bir şekilde tüketicilerin kullanımına sunulması için, rekabet 
ortamı

==== ELEKTRİK PİYASASI DENGELEME VE UZLAŞTIRMA YÖNETMELİĞİ.pdf ====
 
1
 
(14 Nisan 2009 tarihli ve 27200 sayılı Resmi Gazetede yayımlanmıştır.) 
Enerji Piyasası Düzenleme Kurumundan: 
 
ELEKTRİK PİYASASI DENGELEME VE UZLAŞTIRMA YÖNETMELİĞİ 
 
 
BİRİNCİ KISIM 
Genel Hükümler 
BİRİNCİ BÖLÜM 
Amaç, Kapsam, Dayanak ve Tanımlar 
 
 
Amaç 
MADDE 1 – (1) Bu Yönetmeliğin amacı; aktif elektrik enerjisi arz ve talebinin 
dengelenmesine ve uzlaştırmanın ger

In [3]:
import google.generativeai as genai
from dotenv import load_dotenv
import os

load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

models = genai.list_models()

for model in models:
    print(model.name, " - ", model.supported_generation_methods)

models/embedding-gecko-001  -  ['embedText', 'countTextTokens']
models/gemini-1.0-pro-vision-latest  -  ['generateContent', 'countTokens']
models/gemini-pro-vision  -  ['generateContent', 'countTokens']
models/gemini-1.5-pro-latest  -  ['generateContent', 'countTokens']
models/gemini-1.5-pro-001  -  ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro-002  -  ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-pro  -  ['generateContent', 'countTokens']
models/gemini-1.5-flash-latest  -  ['generateContent', 'countTokens']
models/gemini-1.5-flash-001  -  ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-flash-001-tuning  -  ['generateContent', 'countTokens', 'createTunedModel']
models/gemini-1.5-flash  -  ['generateContent', 'countTokens']
models/gemini-1.5-flash-002  -  ['generateContent', 'countTokens', 'createCachedContent']
models/gemini-1.5-flash-8b  -  ['createCachedContent', 'generateContent', 'countToken

In [1]:
import sqlite3
from datetime import datetime
import os
from PyPDF2 import PdfReader
import google.generativeai as genai
from dotenv import load_dotenv
import time

In [2]:
import sqlite3
from datetime import datetime
import os
from PyPDF2 import PdfReader
import google.generativeai as genai
from dotenv import load_dotenv
import time

# ENV ayarları
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
model = genai.GenerativeModel('models/gemini-1.5-flash-latest')

# Veritabanı bağlantısı ve tablo oluşturma
conn = sqlite3.connect("duyurular.db")
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS ozetler (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        dosya_adi TEXT,
        tarih TEXT,
        ozet TEXT
    )
""")
conn.commit()

# PDF'ten metin çıkarma
def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        with open(pdf_path, "rb") as file:
            reader = PdfReader(file)
            for page in reader.pages:
                text += page.extract_text() or ""
    except Exception as e:
        print(f"Hata oluştu dosya okuma sırasında: {e}")
    return text

# Gemini ile özetleme
def summarize_text_with_gemini(text):
    prompt = f"""
    Aşağıdaki elektrik sektörü duyurusunu özetle. Özet kısa, açık ve teknik bir dille yazılsın:
    {text[:12000]}  # Metni token limiti nedeniyle 12.000 karakterle sınırladım.
    """
    retries = 3
    for attempt in range(retries):
        try:
            response = model.generate_content(prompt)
            return response.text.strip()
        except Exception as e:
            print(f"Hata oluştu özetleme sırasında (deneme {attempt+1}): {e}")
            if attempt < retries - 1:
                print("Bir süre bekleniyor...")
                time.sleep(5)
            else:
                print("Tüm denemeler başarısız oldu.")
                return ""

# PDF klasörü
pdf_folder = "indirilenler"

# Klasördeki PDF'leri işle
for file in os.listdir(pdf_folder):
    if file.endswith(".pdf"):
        full_path = os.path.join(pdf_folder, file)
        print(f"İşleniyor: {file}")
        
        text = extract_text_from_pdf(full_path)
        if text:
            summary = summarize_text_with_gemini(text)
            if summary:
                now = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                cursor.execute("INSERT INTO ozetler (dosya_adi, tarih, ozet) VALUES (?, ?, ?)", (file, now, summary))
                conn.commit()
                
                print("Kayıt edildi:", file)
                print("Özet:")
                print(summary)
                print("-" * 50)
            else:
                print(f"Özet alınamadı: {file}")
        else:
            print(f"Metin çıkarılamadı: {file}")
        
        time.sleep(25)

conn.close()


İşleniyor: Elektrik Piyasası Kanunu (6446).pdf
Kayıt edildi: Elektrik Piyasası Kanunu (6446).pdf
Özet:
6446 sayılı Elektrik Piyasası Kanunu (2013), elektriğin yeterli, kaliteli, sürekli, düşük maliyetli ve çevre dostu şekilde tüketime sunulmasını hedefler.  Üretim, iletim, dağıtım, toptan/perakende satış, ithalat/ihracat ve ilgili tüm tarafların hak ve yükümlülüklerini kapsar.  Kanun, bağımsız düzenleme ve denetim mekanizmaları kurarak rekabetçi, mali açıdan güçlü ve şeffaf bir elektrik piyasası oluşturmayı amaçlar.  Kanun,  Enerji Piyasası Düzenleme Kurulu (EPDK) başta olmak üzere, EPİAŞ, EÜAŞ, TEDAŞ, TEİAŞ ve TETAŞ gibi kuruluşların rollerini ve piyasa katılımcılarının (üretim, iletim, dağıtım, tedarik şirketleri, tüketiciler) tanımlarını ve ilişkilerini detaylı şekilde tanımlar.
--------------------------------------------------
İşleniyor: ELEKTRİK PİYASASI DENGELEME VE UZLAŞTIRMA YÖNETMELİĞİ.pdf
Kayıt edildi: ELEKTRİK PİYASASI DENGELEME VE UZLAŞTIRMA YÖNETMELİĞİ.pdf
Özet:
Bu yönetm